# LDA 토픽 모델링
스팸 데이터의 내용과 동향의 양적 분석을 위해 텍스트 마이닝 기법 중 통계 추론 모델인 LDA(Latent Dirichlet Allocation) 토픽 모델링을 사용.


In [ ]:
!pip install konlpy

In [ ]:
!pip install gensim

In [ ]:
!pip install googletrans

In [19]:
#필요한 모듈 임포트
import pandas as pd
from pandas import DataFrame  as df
import numpy as np
from collections import Counter
%matplotlib inline
import matplotlib.pyplot as plt
import re 
import gensim
from gensim import corpora, models
from datetime import datetime
from google.colab import drive

In [20]:
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [21]:
df= pd.read_csv('/content/gdrive/My Drive/빅데이터/Cyber Security/main_df.csv', encoding="utf-8")

In [22]:
df.head()

,MAIL_FROM,Received,Date,text_without_tag,include_url,num_of_imgs,Month,Day,Time,length_of_text,main_language,Group,ip
0,juyoung@gmail.com,from [102.48.236.154] by 73.132.221.32 id <954...,"Mon, 16 Dec 2019 21:41:42 +0100",Content-Type: text/html;\nContent-Transfer-Enc...,0,0,Dec,Mon,21.0,533.0,ko,A,102.48.236.154
1,suminy@outlook.kr,from [167.139.250.4] by 73.132.221.32 with ESM...,"Mon, 20 Apr 2020 22:38:31 -0200",Content-Type: text/html;\nContent-Transfer-Enc...,0,0,Apr,Mon,22.0,509.0,ko,A,167.139.250.4
2,jisun@nate.com,from [211.204.37.3] by 73.132.221.32 with ESMT...,"Mon, 03 Feb 2020 11:41:18 -0600",Content-Type: text/html;\nContent-Transfer-Enc...,0,0,Feb,Mon,11.0,527.0,ko,B,211.204.37.3
3,yenjuy@outlook.kr,from [208.206.157.152] by 73.132.221.32 with E...,"Sun, 05 Apr 2020 22:39:19 +0300",Content-Type: text/html;\nContent-Transfer-Enc...,0,0,Apr,Sun,22.0,217.0,ko,A,208.206.157.152
4,noah@ajne.com,from localhost ([127.0.0.1]) by localhost via ...,"Tue, 21 Apr 2020 17:52:56 +0800",This message contained an attachment that was ...,0,0,Apr,Tue,17.0,210.0,en,D,127.0.0.1


### 언어 번역
google translate API를 사용하여 각 언어를 영어로 번역

# 2-2. 잘 안나오면 샘플 데이터를 전처리 후 번역 진행
구글
### https://medium.com/analytics-vidhya/translate-list-and-pandas-data-frame-using-googletrans-library-in-python-f28b8cb84f21
구글 
### https://py-googletrans.readthedocs.io/en/latest/
XLM-R
### https://towardsdatascience.com/how-to-apply-bert-to-arabic-and-other-languages-5c3410ddd787

# 분석 영역 분류
분석 시간 및 분류를 위해 영역 선택


In [23]:
news_df = pd.DataFrame({'document':df['text_without_tag'], 'language' : df['main_language']})

In [24]:
news_df.head()

,document,language
0,Content-Type: text/html;\nContent-Transfer-Enc...,ko
1,Content-Type: text/html;\nContent-Transfer-Enc...,ko
2,Content-Type: text/html;\nContent-Transfer-Enc...,ko
3,Content-Type: text/html;\nContent-Transfer-Enc...,ko
4,This message contained an attachment that was ...,en


In [25]:
# 결측값 확인
news_df['document'].isnull().sum()

4078

In [26]:
news_df = news_df.fillna('NA')
news_df['document'].isnull().sum()

0

In [27]:
# 전체 단어에 대한 소문자 변환
news_df['document'] = news_df['document'].apply(lambda x: x.lower())

regex_1 = re.compile('^(the original type was:).*|^(the filename was:).*|^(content).*|^(\.footer).*|^\s{0,}(charset).*|\\n',re.M)
regex_2 = re.compile('#body.*}',re.S) 

for i in news_df.index:
    val = news_df._get_value(i, 'document')
    content = re.sub(regex_1,' ',val)
    re_content_v = re.sub(regex_2,' ',content)
    news_df._set_value(i, 'new_document', re_content_v)

### 각 언어별 전처리

In [28]:
for i in news_df.index:
    val = news_df._get_value(i, 'language')
    doc = news_df._get_value(i, 'new_document')
    if val == 'ko' :
        re_data = doc.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ')
        # re_data = clean_Data['new_document'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]',' ',regex=True)
        # clean_Data  = clean_Data.replace(r'^\s*$', None, regex=True)
        news_df._set_value(i, 'new_document', re_data)
        continue
    elif val == 'en':
        re_data = doc.replace("[^a-zA-Z]", " ")   # 특수 문자 제거
        news_df._set_value(i, 'new_document', re_data)
        continue
    elif val == 'ja' :
        re_data = doc.replace("[^\u3040-\u30ff\u3400-\u4dbf\u4e00-\u9fff\uf900-\ufaff\uff66-\uff9f]", " ")
        news_df._set_value(i, 'new_document', re_data)
        continue
    else :
        news_df._set_value(i, 'new_document', doc)
        continue

In [29]:
news_df.head()

,document,language,new_document
0,content-type: text/html;\ncontent-transfer-enc...,ko,안녕하세요. 회원님들~~~! 경기가 위축 되면서 모든게 불확...
1,content-type: text/html;\ncontent-transfer-enc...,ko,안녕하세요.. 카페회원 여러분 mb40 원단이 적용된 품질 좋...
2,content-type: text/html;\ncontent-transfer-enc...,ko,안녕하세요. 온라인 유저님들~~~!!온라인 웹서핑을 하다 보니 ...
3,content-type: text/html;\ncontent-transfer-enc...,ko,안녕하세요 클럽여러분 요즘 스트레스 많이 받으시죠? 스트레스를...
4,this message contained an attachment that was ...,en,this message contained an attachment that was ...


In [ ]:
################### TEST ###################
# 번역
from googletrans import Translator
translator = Translator()
df_sample['Translated_into_English'] = df_sample['text_without_tag'].apply(translator.translate, dest='en').apply(getattr, args=('text',))


In [ ]:
from nltk.corpus import stopwords
import nltk
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from konlpy.tag import Okt
okt = Okt()
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

for i in news_df.index:
    val = news_df._get_value(i, 'language')
    doc = news_df._get_value(i, 'new_document')
    if val == 'ko' :
        tokenized_doc = news_df['new_document'].apply(lambda x: okt.nouns(x)) # 한글 토큰화
        stop_words = '비 분 저 알 절 수 곳 번 사'
        stop_words = stop_words.split(' ')
        tokenized_ko_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
        continue
    elif val == 'en':
        tokenized_doc = news_df['new_document'].apply(lambda x: pos_tag(word_tokenize(x)))
        stop_words = stopwords.words('english')
        tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
        tokenized_en_doc = tokenized_doc.apply(lambda x: [word for word, pos in x if pos in ['NN','NNP']]) # 명사, 고유 명사 선택
        continue
    elif val == 'ja' :
        # tokenized_doc = news_df['new_document'].apply(lambda x: nagisa.tagging(x).words) # 토큰화
        tokenized_ja_doc = news_df['new_document'].apply(lambda x: nagisa.extract(x, extract_postags=['名詞']).words) # 명사만
        continue
    elif val == 'dn' :
        tokenized_doc = news_df['new_document'].apply(lambda x: pos_tag(word_tokenize(x)))
        stop_words = stopwords.words('german')
        tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
        tokenized_dn_doc = tokenized_doc.apply(lambda x: [word for word, pos in x if pos in ['NN','NNP']]) # 'NNG', 'NNP', 'NNB', "NNM"
    else :
        continue

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# 각 언어에 해당하는 토큰 데이터로 LDA 진행
tokenized_en_doc

4                                      [message, multipart]
17        [kkokji, smmobi, date, pharmacy, gift, card, g...
29        [price, viagra, price, today, price, price, pr...
31        [offer, purchase, online, shop, https, johr, p...
100                                                 [plain]
                                ...                        
710373                                              [plain]
710380                                              [plain]
710421    [plain, charset, mime, version, description, m...
710429    [price, cialis, price, today, price, price, pr...
710432    [plain, charset, image, visit, calculus, iegsi...
Name: clean_doc, Length: 54546, dtype: object

### LDA 잠재 디리클레 할당
*   문서의 주제를 알아내는 토픽 모델링의 대표적인 알고리즘
*   LDA는 문서들은 토픽들의 혼합으로 구성되어져 있으며, 토픽들은 확률 분포에 기반하여 단어들을 생성
*   LDA는 각 문서의 토픽 분포와 각 토픽 내의 단어 분포를 추정



###LDA 수행과정
1.   사용자가 알고리즘에 토픽 개수 k를 지정
> - LDA는 토픽의 개수 k를 입력 받으면, k개의 토픽이 문서 전체에 걸쳐 분포되어 있다고 가정
2.   모든 단어를 k개 중 하나의 토픽에 할당
> - 모든 문서의 모든 단어에 대해서 k개 중 하나의 토픽을 랜덤으로 할당
3.   모든 문서의 모든 단어에 대해 다음 사항을 반복. (단, 어떤 문서의 각 단어는 자신은 잘못된 토픽에 할당되어져 있지만, 다른 단어들은 전부 올바른 토픽에 할당되어져 있는 상태라고 가정)
> - 단어는 (문서의 단어들 중 토픽 해당하는 단어들의 비율, 단어를 가지고 있는 모든 문서들 중 토픽이 할당된 비류)두 가지 기준으로 토픽이 재할당











In [ ]:
# 단어에 정수 인코딩을 하는 동시에, 각 문서에서의 단어의 빈도수를 기록
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1])

[(2, 5), (3, 4), (4, 3), (5, 1), (6, 1), (7, 38), (8, 9), (9, 2), (10, 2), (11, 1), (12, 23), (13, 1), (14, 6), (15, 2), (16, 5), (17, 2), (18, 4), (19, 1), (20, 8), (21, 4), (22, 2), (23, 3), (24, 1), (25, 1), (26, 1), (27, 2), (28, 7), (29, 4), (30, 4), (31, 18), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 5), (38, 10), (39, 1), (40, 20), (41, 4), (42, 4), (43, 4), (44, 1), (45, 1), (46, 4), (47, 4), (48, 3), (49, 9)]


In [ ]:
len(dictionary)

100568

In [ ]:
!pip install pyLDAvis

### Topic Coherence 최적의 토픽 수 파악


*   상위 단어 간의 유사도를 계산하여 실제로 해당 주제가 의미론적으로 일치하는 단어들끼리 모여있는지 판단.
*   적당한 step으로 나누어 coherence(주제의 일관성을 측정) 를 계산.
*   높을수록 의미론적 일관성 높음을 의미함으로 최적의 Topic 수를 선정.

In [ ]:
from konlpy.tag import Mecab 
from tqdm import tqdm 
from gensim.models.ldamodel import LdaModel 
from gensim.models.callbacks import CoherenceMetric 
from gensim.models.callbacks import PerplexityMetric 
import logging 
import pickle 
import pyLDAvis.gensim 
from gensim.models.coherencemodel import CoherenceModel 

def compute_coherence_values(dictionary, corpus, texts, start, limit, step): 
    """ Compute c_v coherence for various number of topics Parameters: ---------- dictionary : Gensim dictionary corpus : Gensim corpus texts : List of input texts limit : Max num of topics Returns: ------- model_list : List of LDA topic models coherence_values : Coherence values corresponding to the LDA model with respective number of topics """ 
    coherence_values = [] 
    model_list = [] 
    for num_topics in range(start, limit, step): 
        model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics)  # LDA 모델을 일차적으로 만들고 
        model_list.append(model) 
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')  # CoherenceModel을 통해 최적을 Topic 수를 찾음
        coherence_values.append(coherencemodel.get_coherence())
        coherence = coherencemodel.get_coherence() # Topic의 인과
        print(num_topics)
        print("Coherence", coherence) 
    return model_list, coherence_values

In [ ]:
def find_optimal_number_of_topics(dictionary, corpus, processed_data):
    limit = 30; # 최대 Topic 수 선택 
    start = 3;  # 시작 Topic 수 선택
    step = 2;   # Topic 간격 선택
    model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=corpus, texts=processed_data, start=start, limit=limit, step=step) 
    x = range(start, limit, step) 
    plt.plot(x, coherence_values) 
    plt.xlabel("Num Topics") 
    plt.ylabel("Coherence score") 
    plt.legend(("coherence_values"), loc='best') 
    plt.show()

In [ ]:
# 위 함수들을 통해 그래프 출력 
find_optimal_number_of_topics(dictionary, corpus, tokenized_doc)

### LDA 모델 훈련시키기

기존의 데이터가 총 20개의 카테고리를 가지고 있었으므로 토픽의 개수를 20으로 하여 LDA 모델을 학습

In [ ]:
!pip install pyLDAvis

In [ ]:
import gensim
NUM_TOPICS = 20 # 토픽 수 설정
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4) # 보여줄 단어 개수 선택 
for topic in topics:
    print(topic)

(0, '0.035*"work" + 0.020*"gold" + 0.016*"guide" + 0.013*"information"')
(1, '0.007*"time" + 0.006*"town" + 0.006*"lady" + 0.006*"mario"')
(2, '0.097*"site" + 0.092*"access" + 0.090*"place" + 0.025*"mesg"')
(3, '0.035*"message" + 0.027*"http" + 0.022*"filename" + 0.015*"canvas"')
(4, '0.009*"time" + 0.008*"cricket" + 0.006*"name" + 0.006*"thing"')
(5, '0.013*"time" + 0.007*"woman" + 0.006*"diana" + 0.006*"night"')
(6, '0.073*"energy" + 0.027*"family" + 0.024*"decoration" + 0.019*"newsletter"')
(7, '0.412*"price" + 0.071*"payment" + 0.071*"today" + 0.068*"mastercard"')
(8, '0.012*"time" + 0.008*"redworth" + 0.007*"hand" + 0.007*"woman"')
(9, '0.022*"music" + 0.018*"company" + 0.017*"http" + 0.013*"record"')
(10, '0.014*"barbie" + 0.010*"time" + 0.008*"mattel" + 0.008*"woman"')
(11, '0.020*"system" + 0.015*"range" + 0.014*"body" + 0.013*"instrument"')
(12, '0.014*"time" + 0.010*"world" + 0.009*"woman" + 0.008*"fire"')
(13, '0.151*"width" + 0.054*"decoration" + 0.048*"align" + 0.046*"back

### 토픽 별 단어 분포 확인
* 토픽별 단어의 분포를 확인
* 좌측의 원들은 각 토픽들을 나타냄
* 우측의 단어 그래프에는 Topic 생성에 사용된 단어들을 나타냄
* 좌측의 원중 하나를 선택했을때 우측에 해당 토픽에 대한 예상 용어 빈도가 나타남
* 각 원이 근접 할 수록 유사한 토픽으로 판단


In [ ]:
# 시각화
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

### 문서 별 토픽 분포 확인

In [ ]:
#  이미 훈련된 LDA 모델인 ldamodel[]에 정수 인코딩된 데이터(corpus)를 넣어 각 문서의 토픽 분포 확인
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==20:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(17, 0.9269231)]
1 번째 문서의 topic 비율은 [(5, 0.99709475)]
2 번째 문서의 topic 비율은 [(19, 0.9586956)]
3 번째 문서의 topic 비율은 [(1, 0.5695864), (6, 0.09559023), (11, 0.26943877)]
4 번째 문서의 topic 비율은 [(0, 0.025), (1, 0.025), (2, 0.025), (3, 0.025), (4, 0.025), (5, 0.025), (6, 0.025), (7, 0.025), (8, 0.025), (9, 0.025), (10, 0.025), (11, 0.025), (12, 0.025), (13, 0.025), (14, 0.025), (15, 0.025000002), (16, 0.025), (17, 0.525), (18, 0.025), (19, 0.025)]
5 번째 문서의 topic 비율은 [(4, 0.044518657), (8, 0.09098405), (10, 0.86330175)]
6 번째 문서의 topic 비율은 [(0, 0.025), (1, 0.025), (2, 0.025), (3, 0.025), (4, 0.025), (5, 0.025), (6, 0.025), (7, 0.025), (8, 0.025), (9, 0.025), (10, 0.025), (11, 0.025), (12, 0.025), (13, 0.025), (14, 0.025), (15, 0.025), (16, 0.025), (17, 0.525), (18, 0.025), (19, 0.025)]
7 번째 문서의 topic 비율은 [(0, 0.025), (1, 0.025), (2, 0.025), (3, 0.025), (4, 0.025), (5, 0.025), (6, 0.025), (7, 0.025), (8, 0.025), (9, 0.025), (10, 0.025), (11, 0.025), (12, 0.025), (13, 0.025), (14, 0.

In [ ]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내옴
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장
            else:
                break
    return(topic_table)

In [ ]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 컬럼을 위해서 인덱스 열을 하나 더 만듦
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,17.0,0.9269,"[(17, 0.9269231)]"
1,1,5.0,0.9971,"[(5, 0.99709475)]"
2,2,19.0,0.9587,"[(19, 0.9586956)]"
3,3,1.0,0.5696,"[(1, 0.5696095), (6, 0.09558333), (11, 0.26942..."
4,4,17.0,0.5250,"[(0, 0.025), (1, 0.025), (2, 0.025), (3, 0.025..."
5,5,10.0,0.8633,"[(4, 0.0445161), (8, 0.090988845), (10, 0.8632..."
6,6,17.0,0.5250,"[(0, 0.025), (1, 0.025), (2, 0.025), (3, 0.025..."
7,7,17.0,0.5250,"[(0, 0.025), (1, 0.025), (2, 0.025), (3, 0.025..."
8,8,3.0,0.9620,"[(3, 0.96199995)]"
9,9,11.0,0.9817,"[(11, 0.98173076)]"


In [ ]:
topictable['가장 비중이 높은 토픽'].value_counts().head()

17.0    17430
19.0     8228
3.0      6633
5.0      3432
15.0     2582
Name: 가장 비중이 높은 토픽, dtype: int64

### 문서에서 가장 비중이 높은 Topic Top 2

In [ ]:
print(topics[17])
print(topics[19])
print(topics[3])

(17, '0.083*"plain" + 0.010*"http" + 0.008*"music" + 0.007*"message"')
(19, '0.242*"price" + 0.099*"professional" + 0.099*"viagra" + 0.099*"cialis"')
(3, '0.195*"select" + 0.043*"girls" + 0.042*"want" + 0.042*"find"')
